# Real Estate Sales 2001-2016

The Office of Policy and Management maintains a listing of all real estate sales with a sales price of $2,000 or greater that occur between October 1 and September 30 of each year. For each sale record, the file includes: town, property address, date of sale, property type (residential, apartment, commercial, industrial or vacant land), sales price, and property assessment.

Source: <https://data.ct.gov/Housing-and-Development/Real-Estate-Sales-2001-2016/5mzw-sjtu>

**INPUT**
Each row is an individual sale that includes address, assessed value and sale price

**OUTPUT**
The processed dataset should contain aggregated counts per town per price bucket (<100k, <200k, <300k, <400k, 400k+)

**NOTES** We only care about Single Family homes, without ay Non-use codes applied to them

In [83]:
import pandas as pd
import csv

In [6]:
raw = pd.read_csv('../raw/Real_Estate_Sales_2001-2016.csv', index_col=0, dtype=str)

In [125]:
town2fips = pd.read_csv('https://raw.githubusercontent.com/CT-Data-Collaborative/ct-town-county-fips-list/master/ct-town-county-fips-list.csv', dtype=str, index_col=0)

### Extract only relevant sales from the dataset

In [105]:
# Single Family homes with Non-Use code = 0
relevant = raw.copy(deep=True)
relevant['NonUseCode'] = relevant['NonUseCode'].astype(str).fillna('0')

relevant = relevant[
    (relevant['ResidentialType'] == 'Single Family')
    & (relevant['NonUseCode'] == '0' )
]

# Sale Amount should be a positive float
relevant['SaleAmount'] = relevant['SaleAmount'].astype(float)
relevant = relevant[relevant['SaleAmount'] > 0]

# Remove all unnecessary columns
relevant = relevant.filter(['ListYear', 'Town', 'SaleAmount'])

relevant['Price Range'] = pd.cut(
    relevant['SaleAmount'],
    [0, 100000, 200000, 300000, 400000, 10000000000],
    labels=[ 'Less than $100,000',
             '$100,000 to $199,999',
             '$200,000 to $299,000',
             '$300,000 to $399,000',
             '$400,000 and Over']
)

In [134]:
relevant_counts = relevant.groupby(['Town', 'ListYear', 'Price Range']).size()
relevant_counts = relevant_counts.unstack(level=0,fill_value=0).stack().reset_index()

# Calculate Counties counts and append to the main dataframe
relevant_counts['County'] = relevant_counts['Town'].apply(lambda t: town2fips.loc[t]['County'])

counties = relevant_counts.groupby(['County', 'ListYear', 'Price Range']).sum()
counties = counties.unstack(level=0,fill_value=0).stack().reset_index()
counties['Town'] = counties['County'].apply(lambda c: c + ' County')

# Calculate CT values
ct = relevant_counts.groupby(['ListYear', 'Price Range']).sum()
#print(relevant_)
ct = ct.unstack(level=0,fill_value=0).stack().reset_index()
ct['Town'] = 'Connecticut'

relevant_counts_combined = pd.concat([relevant_counts, counties, ct], sort=False)

# Add remaining columns
relevant_counts_combined['FIPS'] = relevant_counts_combined['Town'].apply(lambda t: town2fips.loc[t]['FIPS'])
relevant_counts_combined['Measure Type'] = 'Number'
relevant_counts_combined['Variable'] = 'Number of Home Sales'

relevant_counts_combined.to_csv('../data/single-family-home-sales-2001-2016.csv', index=False,
                       columns=['Town', 'FIPS', 'ListYear', 'Price Range', 'Measure Type', 'Variable', 0],
                       header=['Town/County', 'FIPS', 'Year', 'Price Range', 'Measure Type', 'Variable', 'Value'],
                       quoting=csv.QUOTE_NONNUMERIC)

In [132]:
relevant_counts.groupby(['ListYear', 'Price Range']).sum()#.count()

0
ListYear Price Range                
2001     Less than $100,000     1776
         $100,000 to $199,999  11148
         $200,000 to $299,000   6450
         $300,000 to $399,000   3258
         $400,000 and Over      4938
2002     Less than $100,000     2464
         $100,000 to $199,999  18812
         $200,000 to $299,000  12407
         $300,000 to $399,000   6450
         $400,000 and Over     10524
2003     Less than $100,000     1192
         $100,000 to $199,999   9088
         $200,000 to $299,000   7999
         $300,000 to $399,000   4320
         $400,000 and Over      6926
2004     Less than $100,000     1091
         $100,000 to $199,999   8984
         $200,000 to $299,000  11047
         $300,000 to $399,000   6198
         $400,000 and Over     11258
2005     Less than $100,000      636
         $100,000 to $199,999   5796
         $200,000 to $299,000   8911
         $300,000 to $399,000   4851
         $400,000 and Over      7640
2006     Less than $100,000      334
         $100,000 to $199,999   3533
         $200,000 to $299,000   7130
         $300,000 to $399,000   4433
         $400,000 and Over      7651
...                              ...
2011     Less than $100,000      392
         $100,000 to $199,999   3186
         $200,000 to $299,000   3385
         $300,000 to $399,000   1981
         $400,000 and Over      4386
2012     Less than $100,000      437
         $100,000 to $199,999   4005
         $200,000 to $299,000   4476
         $300,000 to $399,000   2709
         $400,000 and Over      4684
2013     Less than $100,000      530
         $100,000 to $199,999   4535
         $200,000 to $299,000   4672
         $300,000 to $399,000   2721
         $400,000 and Over      4679
2014     Less than $100,000     7600
         $100,000 to $199,999  12896
         $200,000 to $299,000   9115
         $300,000 to $399,000   5230
         $400,000 and Over      8745
2015     Less than $100,000     1757
         $100,000 to $199,999   8455
         $200,000 to $299,000   7728
         $300,000 to $399,000   4310
         $400,000 and Over      5611
2016     Less than $100,000     1789
         $100,000 to $199,999   8264
         $200,000 to $299,000   7994
         $300,000 to $399,000   4726
         $400,000 and Over      7134

[80 rows x 1 columns]